# Computing
## Neural Networks

In [1]:
library(AppliedPredictiveModeling)
data(solubility)
ls(pattern="^solT") 

[1] "solTestX"       "solTestXtrans"  "solTestY"       "solTrainX"     
[5] "solTrainXtrans" "solTrainY"

In [ ]:
library(caret)
cor_info <- findCorrelation(cor(solTrainXtrans),cutoff = 0.75)
trn <- solTrainXtrans[,-cor_info]
vld <- solTestXtrans[,-cor_info]
grid_nnet <- expand.grid(.decay = c(0,0.01,0.1),
                        .size = c(1:10),
                        .bag=F)
set.seed(100)
fit_nnet <- train(trn,solTrainY,method = "avNNet",tuneGrid = grid_nnet,
                 preProc=c("center","scale"),linout=T,trace=F,
                 MaxNWts=10*(ncol(trn)+1)+10+1,maxit=500)

Warning message:
"executing %dopar% sequentially: no parallel backend registered"

In [ ]:
fit_nnet
str(fit_nnet)

## MARS

In [ ]:
library(earth)
fit_mars <- earth(solTrainXtrans,colTrainY)
fit_mars
summary(marsFit)
plotmo(fit_mars)

In [ ]:
#caret wrapper
grid_mars <- expand.grid(.degree=1:2,.nprune=2:38)
set.seed(100)
fit_mars <- train(solTrainXtrans,solTrainY,
                 method="earth",tuneGrid=marsGrid,trControl(methon="cv"))
fit_mars

In [ ]:
head(predict(fit_mars,solTestXtrain))

In [ ]:
varImp(fit_mars)

## SVM

In [ ]:
library(kernlab)
fit_svm <- ksvm(x=solTrainXtrans,y=solTrainY,kernel="rbfdot",kpar="automatic",
               C=1,epsilon=0.1)
fit_svm
#kernel = "polydot "多项式核函数
#kernel = "vanilladot " 线性核函数

In [ ]:
fit_svm <- train(solTrainXtrans,solTrainY,method="svmRadial",
                preProc=c("center","scale"),tuneLength=14,trControl=trainControl(methon="cv"))
fit_svm$finalModel

## knn

In [ ]:
#Remove a few sparse and unbalanced fingerprints first
trn <- solTrainXtrans[,-nearZeroVar(solTrainXtrans)]
set.seed(100)
fit_knn <- train(trn,solTrainY,method="knn",
                preProc=c("center","scale"),
                tuneGrid=data.frame(.k=1:20),
                trControl=trainControl(method="cv"))

fit_knn